In [1]:
import tweepy
import json
import pandas as pd
import random
from dotenv import load_dotenv
import os
import time
from datetime import datetime
load_dotenv()

True

# Sentiment analysis de tweets respecto al plebiscito en Septiembre 2022 Chile

Este script busca en tweeter, con la los privilegios de la cuenta develop gratuita, los tweets según las palabras clave, la localización y las fechas determinadas.

La información mas importante de cada tweet es guardado en un dataframe.

La API libre de twitter tiene varias limitaciones, entre ella la fecha:

    Keep in mind that the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week.
    
No se puede traer tweets mas antiguos que una semana. Ademas del peso sobre la cantidad de tweets, que cuelga la API por 15 minutos

##### Definición de palabras clave (_keywords_) a buscar dentro del texto de los tweets:

In [39]:
keywords_list = ['Constituyente',
'Convención Constituyente',
'Convención Constitucional',
'Asamblea Constituyente',
'plebiscito',
'referéndum',
'constitución chilena',
'Acuerdo por la paz social y la Nueva Constitución',
'Vamos por Chile',
'Unión Demócrata Independiente',
'Constituyentes Apartidistas',
'Independientes No Neutrales',
'Lista del Pueblo',
'Apruebo Dignidad',
'Lista del Apruebo',
'Pueblo Constituyente']

keywords_list_II = ['Chile Digno',
'Chile Unido',
'Unidos por Chile'
'Chile Libre',
'Coordinadora Constituyente Plurinacional y Popular',
'Somos región',
'Colectivo Socialista',
'Independientes por una Nueva Constitución',
'Movimientos sociales constituyentes',
'Colectivo del Apruebo',
'Wallmapur',
'Elisa Loncón',
'Campaña de desprestigio',
'rechazo del prebiscito de 2020',
'Carmen Gloria Valladares Moyano',
'Elisa Loncon',
'Destitucion de Elisa Loncon']

In [40]:
keywords_list = ["(" + word + ")" for word in keywords_list]

'#DestitucionDeElisaLoncon',
'juicio político',
'Jaime Bassa',
'Marcela Cubillos',
'Jaime Mañalich',
'Autodesprestigio',
'María Elisa Quinteros Cáceres',
'Gaspar Dominguez Donoso']

In [41]:
keywords_list  = ' OR '.join(keywords_list)

In [42]:
keywords_list

'(Constituyente) OR (Convención Constituyente) OR (Convención Constitucional) OR (Asamblea Constituyente) OR (plebiscito) OR (referéndum) OR (constitución chilena) OR (Acuerdo por la paz social y la Nueva Constitución) OR (Vamos por Chile) OR (Unión Demócrata Independiente) OR (Constituyentes Apartidistas) OR (Independientes No Neutrales) OR (Lista del Pueblo) OR (Apruebo Dignidad) OR (Lista del Apruebo) OR (Pueblo Constituyente)'

#### Funciones
El script tiene dos funciones principales:
- Activación de la API con las credenciales
- Búsqueda de los tweets con lo parámetros definidos

##### Función Activación

In [34]:
# Función de generación de la API
def oauth():
    #Pass in our twitter API authentication key
#     consumer_key = os.getenv('CONSUMER_KEY')
#     consumer_secret = os.getenv('CONSUMER_SECRET')
#     access_token = os.getenv('ACCESS_TOKEN')
#     access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')
    
    auth = tweepy.OAuth2BearerHandler('BEARER_TOKEN')
    api = tweepy.API(auth)
#     auth = tweepy.OAuth1UserHandler(
#         consumer_key, consumer_secret,
#         access_token, access_token_secret
#     )
    #Instantiate the tweepy API
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

##### Función Búsqueda

In [21]:
# Función de búsqueda de tweets según los keywords, cantidad y localización
def search(geo, search_query, page_limit):
    page_no = 1
    # Creamos las columnas para el DataFrame
    data = []
    columns = ['Time','User', 'Tweet']
    try:
        # Lanzamos la búsqueda
        pages = tweepy.Cursor(api.search_tweets,
                              q=search_query, 
                              tweet_mode='extended', 
                              geocode = geo).pages(page_limit)

        # Iteramos por cada una de las páginas de resultados, y agregamos los resultados a una lista
        for page in pages:    
            for i in range(len(page)):
                data.append([page[i].created_at, page[i].user.screen_name, page[i].full_text])
            page_no += 1
        
        # Volcamos los datos al DataFrame
        df = pd.DataFrame(data, columns=columns) 
        
        return df
    
    except BaseException as e:
        print('Status Failed On,',str(e))
        return

#### Ejecución 

##### Instanciamos la API

In [9]:
api = oauth()

#####  Cargamos los parámetros de búsqueda


- `sgo`: latitud y longitud de Santiago de Chile
- `radio`: radio en km alrededor del punto
- `no_of_tweets`: Cantidad de tweets para cada keyword
- `geo`: arma el feature según lo cargado anteriormente, con la ciudad y el radio.
- `keywords_list`: Se incluye en esta sección sólo para testear. Se limita a dos palabras. Si se comenta esa línea, usa toda la lista presentada arriba)

In [50]:
sgo = '-33.4489,-70.6693'
radio = '100km'
page_limit =50
geo = str(sgo) + ',' + str(radio) 
# keywords_list = ['Chile'] # Comentar esta linea para usar toda la keywords_list


In [51]:
keywords_list

'(Constituyente) OR (Convención Constituyente) OR (Convención Constitucional) OR (Asamblea Constituyente) OR (plebiscito) OR (referéndum) OR (constitución chilena) OR (Acuerdo por la paz social y la Nueva Constitución) OR (Vamos por Chile) OR (Unión Demócrata Independiente) OR (Constituyentes Apartidistas) OR (Independientes No Neutrales) OR (Lista del Pueblo) OR (Apruebo Dignidad) OR (Lista del Apruebo) OR (Pueblo Constituyente)'

#####  Lanzamos la búsqueda

In [52]:
# df = pd.DataFrame()
df = search(geo,
            keywords_list,
            page_limit)


In [53]:
pd.set_option('display.max_colwidth', None)
df

,Time,User,Tweet
0,2022-08-24 17:31:16+00:00,umana_patricio,"@melnicksergio El elenco del r3ch4z0 que en la franja prometen una escrita con amor...\nPero la verdad son solo odio...\nAcá le dejo las palabras de Rocío cantuarias, constituyente del r3ch4z0... https://t.co/jRkRb4cYJX"
1,2022-08-24 17:30:39+00:00,Celia_Alejandra,"El presidente sólo cumple con su obligación de llamar a participar en el plebiscito, pues ambas opciones son igual de válidas (🤥). https://t.co/5D1n78bWu2"
2,2022-08-24 17:29:26+00:00,norasussy,@carreragonzalo @libre_porchile Los empresarios desconocen el real y aplastante objetivo y poder de los indigenistas q recibirán desde el texto constituyente?\nTendrán Escaños Reservados en todos los organismos nacionales:\nCám. d Diput.\nCám. d las regiones\nConsejo Municipal\nComi. d Fiscaliz d E* d Excepc.\nAC.?
3,2022-08-24 17:29:03+00:00,RernstDe,@Jacquel53524526 @AlvaroB16162393 @hernan_sr En principio claro. Pero hoy me es sospechoso. A semanas de un plebiscito derrepente ahora si?
4,2022-08-24 17:28:57+00:00,jpiquer,"@MikevanTreek @Jicahis El problema es el NO cambio de hora, que es el día antes del plebiscito y dónde muchos relojes pueden cambiar la hora por error."
...,...,...,...
745,2022-08-23 20:51:22+00:00,victorar1966,Aquí en twitter nos hemos alejado del foco de este plebiscito. El 4/S se vota si se acepta esta propuesta de NC o se rechaza. Así de simple. Que si este es feo o que robo o que es flojo.....eso no corre.. Apruebe o Rechace.
746,2022-08-23 20:49:11+00:00,finco,"@SenadoresDC @ximerincon @matiaswalkerp @24HorasTVN @CNNChile @CHVNoticias @meganoticiascl @T13 @biobio wajajaja dos semanas antes del plebiscito, DOS SEMANAS, no les da verguenza. #AprueboUnChileMejor"
747,2022-08-23 20:47:23+00:00,rodosauriorex,@fireman331 @rafagotelli @DFinanciero Y cual es el objetivo de reactivarlo justo ahora?? Solo busca subir un par de puntos de aprobación de cara al plebiscito. Gobietno inútil y populista.
748,2022-08-23 20:47:23+00:00,LuchadoraSocial,@mariacris1708 ¿Y el no es político? Tengo entendido que fue en la lista de Apruebo Dignidad en cupo de Convergencia Social y salió electo por gestión de un partido político. Cuidado.


#####  Guardamos el dF

In [ ]:
filename = 'plebiscito_' + datetime.now().strftime('%m_%d_%Y_%H:%M:%S') + '.csv'
df.to_csv(filename, index = False)